In [ ]:
import holoviews as hv
import numpy as np
hv.extension('bokeh')
import pandas as pd
from scipy import stats

In [ ]:
%opts Curve [width=600 height=400] (color='black')
%opts Curve.reference (color='green', line_dash='dotted')
%opts Area [height=400] (line_width=2 alpha=.5) VLine (color='red') {+framewise}

# Normal Distrubtion

In [ ]:
xs = np.linspace(-3, 3, 600)

def function(xs, mu, sigma):
    "Some time varying function"
    return stats.norm(loc=mu, scale=sigma).pdf(xs)

def integral(limit, mu, sigma):
    limit = -3 if limit is None else np.clip(limit,-3,3)
    if limit > 0:
        curve = hv.Curve((xs, function(xs, mu, sigma)))[:limit]
        area  = hv.Area ((xs, function(xs, mu, sigma)))[limit:]
    else:
        curve = hv.Curve((xs, function(xs, mu, sigma)))[limit:]
        area  = hv.Area ((xs, function(xs, mu, sigma)))[:limit]
        
    summed = area.dimension_values('y').sum() * 0.015  # Numeric approximation
    return (area * curve * hv.VLine(limit) * hv.Text(limit + 0.5, .20, 'p = %.3f' % summed))

# hv.DynamicMap(integral, streams=[hv.streams.Stream.define('Sigma', time=1.0)(), 
#                                  hv.streams.PointerX().rename(x='limit')])

hv.DynamicMap(
    integral, 
    kdims=['mu','sigma'],
    streams=[ 
        hv.streams.PointerX().rename(x='limit')
    ]
).redim.range(
    mu=(-1., 1.),
    sigma=(.2, 1.))


# Students T Distribution

In [ ]:
xs = np.linspace(-3, 3, 600)

def function(xs, mu, sigma ,dof):
    return stats.t(df=dof, loc=mu, scale=sigma).pdf(xs)

def reference(xs, mu, sigma):
    return stats.norm(loc=mu, scale=sigma).pdf(xs)

def integral(limit, mu, sigma, dof):
    limit = -3 if limit is None else np.clip(limit,-3,3)
    if limit > 0:
        curve = hv.Curve((xs, function(xs, mu, sigma, dof)))[:limit]
        area  = hv.Area ((xs, function(xs, mu, sigma, dof)))[limit:]
    else:
        curve = hv.Curve((xs, function(xs, mu, sigma, dof)))[limit:]
        area  = hv.Area ((xs, function(xs, mu, sigma, dof)))[:limit]
        
    summed = area.dimension_values('y').sum() * 0.015  # Numeric approximation
    out = (area * curve * hv.VLine(limit) * hv.Text(limit + 0.5, .20, 'p = %.3f' % summed))
    out = out * hv.Curve((xs, reference(xs, mu, sigma)), group='reference')
    return out

hv.DynamicMap(
    integral, 
    kdims=['mu','sigma', 'dof'],
    streams=[ 
        hv.streams.PointerX().rename(x='limit')
    ]
).redim.range(
    mu=(-1., 1.),
    sigma=(.1, 1),
    dof=(2, 50)
)
